<a href="https://colab.research.google.com/github/Drake-HeleneVeenstra/workshops/blob/main/MLworkshop_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Klustering algoritmer för segmentering - från statistik till maskininlärning**

**1. Som förberedning för klusteringanalys behöver vi ladda in vissa libraries samt skapa data.**
Dessa libraries innehåller visualiseringsmöjligheter (matplotlib och seaborn) och de faktiska clusterings algoritmer (sklearn.cluster, hdbscan). Numpy underlättar datamanipulering, och sklearn.datasets ger möjlighet att skapa data (liknande 'load inline' in Qlik).

Sen skapar vi data och plottar den. Du kan ändra parametrar i moons och blobs för att ändra datans utseende.

In [ ]:
# 1 - skapa dataset och scatterplot

# installation behövs för att kunna köra hdbscan
!pip install hdbscan

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster as cluster
import sklearn.datasets as data
import hdbscan
plot_kwds = {'alpha': 0.5, 's': 80, 'linewidths': 0}

# Här skapar vi data, i form av månar och blobs. Ändra n_samples, noise, centers, cluster_std efter behag
moons, _ = data.make_moons(n_samples=100, noise=0.05)
blobs, _ = data.make_blobs(n_samples=100, centers=[(1, 2.25), (-1.0, 1.9)], cluster_std=0.2)
datastack = np.vstack([moons, blobs])

# Plot:
plt.scatter(datastack.T[0], datastack.T[1], c='b', **plot_kwds)
plt.title('1 Scatterplot of generated data', fontsize=20)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)

**2a. Vår första algoritm blir en K-means klustering algoritm.** I kwargs-argumentet finns några utvalda hyperparameters definierad som påverkar utfall av klustering. 'n_clusters' bestämmer i hur många kluster datasetet blir uppdelat. 'random_state' bestämmer random seed som utgångspunkt. Att definiera denna betyder samma resultat när man skulle testa algoritmen igen nästa vecka.  Testa att ändra antal clusters i variabeln 'kwargs' from 5 till något annat.

In [ ]:
# 2a - K-means clustering on dataset

kwargs = {'n_clusters': 4, 'random_state': 38}

algorithm = cluster.KMeans
labels = algorithm(**kwargs).fit_predict(datastack)
palette = sns.color_palette('muted', np.unique(labels).max() + 1)
colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
plt.scatter(datastack.T[0], datastack.T[1], c=colors)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)
plt.title('2a Clustering method: {}'.format(str(algorithm.__name__)),
          fontsize=20);

>K-means är en partitioning algoritm som har som antagandet att det finns globala partitions som är centroid-based. Denna algoritm behöver input om hur många klusters det finns att hitta i datasettet. Prestanda på denna dataset är dålig, den grupperar och segrererar tydlig felaktig, för den kan inte fånga en måna som en segment.

**2b. Vi tar en närmare titt på hur man kan modifiera andra hyperparametrar än antal kluster för att komma närmare verkligheten.** För det skapar vi ett andra dataset som består utav tre blobs, två nära varandra och en blob med stor spridning längre bort. I denna figur har vi inte än applicerat en klusteringalgoritm, färgerna i scatterplot visar bara hur de tre originalblobs är definierat.

In [ ]:
# 2b - skapa ny dataset med spridda kluster, och plotta de tre blobs i olika färger
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster as cluster
import sklearn.datasets as data
plot_kwds = {'alpha': 0.5, 's': 80, 'linewidths': 0}

# Nu skapar vi bara blobs (inga moons), med större spridning
blobs_tmp_1, _ = data.make_blobs(n_samples=300, centers=[(1.2, 2.25)], cluster_std=0.4)
blobs_tmp_2, _ = data.make_blobs(n_samples=300, centers=[(2, 1)], cluster_std=0.4)
blobs_tmp_3, _ = data.make_blobs(n_samples=300, centers=[(20, 5)], cluster_std=5)
datastack_tmp = np.vstack([blobs_tmp_1, blobs_tmp_2, blobs_tmp_3])

# Plot:
plt.scatter(blobs_tmp_1.T[0], blobs_tmp_1.T[1], c='b', **plot_kwds)
plt.scatter(blobs_tmp_2.T[0], blobs_tmp_2.T[1], c='r', **plot_kwds)
plt.scatter(blobs_tmp_3.T[0], blobs_tmp_3.T[1], c='g', **plot_kwds)
plt.title('2b Scatterplot of generated data',fontsize=20)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)

**2c. Applicering av en K-means klustering likadant som i 2a**

In [ ]:
# 2c - K-means algoritm

kwargs_tmp = {'n_clusters': 3, 'random_state': 38, 'init': 'random'}

algorithm = cluster.KMeans
labels = algorithm(**kwargs_tmp).fit_predict(datastack_tmp)
palette = sns.color_palette('muted', np.unique(labels).max() + 1)
colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
plt.scatter(datastack_tmp.T[0], datastack_tmp.T[1], c=colors)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)
plt.title('2c Clustering method: {}'.format(str(algorithm.__name__)),
          fontsize=20);

**2d.** Det syns tydlig i figuren 2c ovan att klustering blir fel, stora spridda blobben är egentligen en blob och borde falla i en kluster/segment. De två små blobs till vänster blir sammanslagen i en och samma segment.
Dock har vi kunskap om vår data, vi vet ju vart blobbens centroid ligger. I riktig data har vi inte exakt information om det, men vi kan ha ungefärlig beskrivning av en centroid. **Vi kan lägga centroids som utgångsinformation för klusterdefinition i modellen:**




In [ ]:
# 2d - vi definierar centroids som vi hade kunskap om som utgångspunkt av klusterdefinition ('init')

centers=np.asarray([[1, 2.25], [2, 1], [20, 5]])
kwargs_tmp = {'n_clusters': 3, 'random_state': 38, 'init': centers, 'n_init': 1}

algorithm = cluster.KMeans
labels = algorithm(**kwargs_tmp).fit_predict(datastack_tmp)
palette = sns.color_palette('muted', np.unique(labels).max() + 1)
colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
plt.scatter(datastack_tmp.T[0], datastack_tmp.T[1], c=colors)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)
plt.title('2d Clustering method: {}'.format(str(algorithm.__name__)),
          fontsize=20);

>Det finns fler sätt att optimera K-means kluster. T.ex. kan du gå tillbaka till kod i 2c och testa att separera i fler än tre kluster. Blev det bättre resultat?

**3.** De månar i vårt originaldataset går inte att separera med en K-means klustering. Vi går vidare och testar flera algoritmer, och ändra några av deras hyperparametrar för att se huruvida dessa går att optimera. **Vi testar en Affinity Propagation algoritm näst**:

In [ ]:
# 3 - Affinity Propagation
kwargs = {'preference': -6.0, 'damping': 0.85}

algorithm = cluster.AffinityPropagation
labels = algorithm(**kwargs).fit_predict(datastack)
palette = sns.color_palette('muted', np.unique(labels).max() + 1)
colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
plt.scatter(datastack.T[0], datastack.T[1], c=colors)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)
plt.title('3 Clustering method: {}'.format(str(algorithm.__name__)),
          fontsize=20);

>Affinity Propagation är en graph-baserad point-voting system, följd av en centroid-baserad partitioning approach. Fördelen med den över K-means är att man inte behöver bestämma på förhand hur många segmenter det ska hittas. Nackdelen är att den har inte bättre performance än K-means i denna dataset. Dessutom är de hyperparametrar i denna modell inte lätt-optimerad, dessutom är algoritmen långsammare än K-means. Eftersom denna modell förbättrar inte vårt segmentering jämfört med K-means så diskuterar vi inte den i djupet men går vi vidare.

**4. Mean Shift clustering. Igen en centroid-baserad algoritm. Denna algoritm gör riktig klustering snarare än partitioning.** Skillnaden är att en partitioning algoritm fungerar som en top-down gruppering av alla datapunkter, vilket föredrar en mindre antal större grupper. Partitioning börjar istället bottom-up, och utgår från konnektiviteten mellan punkter. Om vi väljer 'cluster_all': False så visas att inte alla punkter rimligtvis tillhör någon segment. Ändra till True för att tvinga alla punkter att falla i en segment, och se att det försämras segmenteringen. 

>

In [ ]:
# 4 - Mean Shift clustering
kwargs = {'cluster_all': False}

algorithm = cluster.MeanShift
labels = algorithm(0.3, **kwargs).fit_predict(datastack)
palette = sns.color_palette('muted', np.unique(labels).max() + 1)
colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
plt.scatter(datastack.T[0], datastack.T[1], c=colors)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)
plt.title('4 Clustering method: {}'.format(str(algorithm.__name__)),
          fontsize=20);

Vi har fortfarande inte sett en tydlig förbättring i prestanda av modellen. Det finns fler algoritmer att testa, dock de kommer inte att göra en förbättring för just den dataset. Om du vill kan du dock testa i kod 2a de följande algoritmer genom att substituera cluster.KMeans med cluster.AgglomerativeClustering eller cluster.SpectralClustering; välj antal kluster i kwargs med 'n_cluster' och ta bort 'random_state' parameter. Vi fortsätter här med någonting helt annorlunda:

5. DBSCAN står för density-based spatial clustering of application with noise. Som namnet antyder gör den en klustering baserat på density; hur tight punkter ligger tillsammans med sina grannar. DBSCAN visar ett tydlig överlägsen prestanda jämfört med de tidigare modeller. eps (epsilon) är en parameter som beskriver grannområdet från varje datapunkt modellen väljer som potentiell startpunkt för en kluster. min_samples står för minimal antal punkter som beskriver en 'dense region', alltså hur många punkter tight tillsammans är trovärdigt för dig som utgångspunkt (core point) för en kluster. Testa olika värden på dessa hyperparametrar i kwargs för att påverka klustering.

In [ ]:
kwargs = {'eps': 0.25, 'min_samples': 5}

algorithm = cluster.DBSCAN
labels = algorithm(**kwargs).fit_predict(datastack)
palette = sns.color_palette('muted', np.unique(labels).max() + 1)
colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
plt.scatter(datastack.T[0], datastack.T[1], c=colors)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)
plt.title('Clustering method: {}'.format(str(algorithm.__name__)),
          fontsize=20);

6. Vi har sparat det bästa för sist. HDBSCAN är en optimerad DBSCAN, h står för hierarchical. Alla punkter har en attribut vilket beskriver hur starkt membership till en viss kluster är. Så vi kan plotta det som varierande saturation, för att visa skillnad mellan core och ytterkanten av en kluster. Detta är direkt användbart om man tänker riktig data; för nu kan vi visa skillnaden mellan kunder / produkten som är typiskt för en segment jämfört med kunder/produkter som kan tillhöra en segment men har vissa avvikande kännetecken.

In [ ]:
clusterer = hdbscan.HDBSCAN(algorithm='best', alpha=1.0,
                            approx_min_span_tree=True,
                            gen_min_span_tree=True, leaf_size=40,
                            metric='euclidean', min_cluster_size=6,
                            min_samples=None, p=None,
                            cluster_selection_method='eom')
clusterer = clusterer.fit(datastack)
palette = sns.color_palette('deep')
cluster_colors = [sns.desaturate(palette[col], sat)
                  if col >= 0 else (0.5, 0.5, 0.5) for col, sat in
                  zip(clusterer.labels_, clusterer.probabilities_)]
plt.scatter(datastack.T[0], datastack.T[1], c=cluster_colors, **plot_kwds)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)
plt.title('Clustering method: {} (leaf)'.format(str(hdbscan.HDBSCAN.__name__)),
          fontsize=20);

Om du vill leka mer så är det följande att rekommendera:
* Gå till https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster och hitta olika algoritmer, du kan ta en algoritm diskuterat här och lägga till fler hyperparametrar för finetuning.

* Ta datasettet från början och modulera den. Annorlunda dataset ger annorlunda prestanda av olika modeller. Om datasetet är enkelt så räcker det med enkel klustering. Dock du kommer att märka att (H)DBSCAN har överlägset prestanda och är ett bra klusteringval.